In [0]:
CREATE OR REFRESH STREAMING LIVE TABLE 03_gold.bridge_metrics
COMMENT "10-min avg temperature, max vibration & max tilt per bridge with window start/end"
AS
WITH
-- Temperature aggregation (10-min avg)
temp_agg AS (
  SELECT
      bridge_id,
      name,
      location,
      window.start AS window_start,
      window.end   AS window_end,
      AVG(temperature) AS avg_temperature
  FROM STREAM(LIVE.02_silver.bridge_temperature)
  WATERMARK event_time DELAY OF INTERVAL 2 MINUTES
  GROUP BY
      window(event_time, '10 minutes'),
      bridge_id,
      name,
      location
),

-- Vibration aggregation (10-min max)
vib_agg AS (
  SELECT
      bridge_id,
      window.start AS window_start,
      window.end   AS window_end,
      MAX(vibration) AS max_vibration
  FROM STREAM(LIVE.02_silver.bridge_vibration)
  WATERMARK event_time DELAY OF INTERVAL 2 MINUTES
  GROUP BY
      window(event_time, '10 minutes'),
      bridge_id
),

-- Tilt aggregation (10-min max)
tilt_agg AS (
  SELECT
      bridge_id,
      window.start AS window_start,
      window.end   AS window_end,
      MAX(tilt_angle) AS max_tilt_angle
  FROM STREAM(LIVE.02_silver.bridge_tilt)
  WATERMARK event_time DELAY OF INTERVAL 2 MINUTES
  GROUP BY
      window(event_time, '10 minutes'),
      bridge_id
)

-- Final join
SELECT
    t.bridge_id,
    t.name,
    t.location,
    t.window_start,
    t.window_end,
    ROUND(t.avg_temperature, 2) AS avg_temperature,
    v.max_vibration,
    l.max_tilt_angle
FROM temp_agg t
INNER JOIN vib_agg v
  ON t.bridge_id = v.bridge_id
 AND t.window_start = v.window_start
 AND t.window_end   = v.window_end
INNER JOIN tilt_agg l
  ON t.bridge_id = l.bridge_id
 AND t.window_start = l.window_start
 AND t.window_end   = l.window_end;
